# Simple Convnet on the MNIST Dataset

The notebook can check GPU linkage for nvidia-docker2 images.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.

Ref: https://raw.githubusercontent.com/fchollet/keras/master/examples/mnist_cnn.py

## GPU Connection Checking

In [1]:
! nvidia-smi

Sun Oct 14 15:26:15 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.48                 Driver Version: 410.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0  On |                  N/A |
| 28%   44C    P0    65W / 250W |    443MiB / 11177MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6107984220332159104, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10470503220
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5763638054878960419
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
import tensorflow
print(tensorflow.test.is_built_with_cuda())

True


## Convnet by Keras

In [4]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [5]:
batch_size = 128
num_classes = 10
epochs = 12

In [6]:
# input image dimensions
img_rows, img_cols = 28, 28

In [7]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [8]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [10]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [11]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 4s 58us/step - loss: 0.2761 - acc: 0.9141 - val_loss: 0.0627 - val_acc: 0.9795
Epoch 2/12
60000/60000 [==============================] - 3s 52us/step - loss: 0.0947 - acc: 0.9724 - val_loss: 0.0444 - val_acc: 0.9854
Epoch 3/12
60000/60000 [==============================] - 3s 50us/step - loss: 0.0678 - acc: 0.9798 - val_loss: 0.0380 - val_acc: 0.9873
Epoch 4/12
60000/60000 [==============================] - 3s 47us/step - loss: 0.0554 - acc: 0.9833 - val_loss: 0.0315 - val_acc: 0.9892
Epoch 5/12
60000/60000 [==============================] - 3s 54us/step - loss: 0.0481 - acc: 0.9853 - val_loss: 0.0300 - val_acc: 0.9900
Epoch 6/12
60000/60000 [==============================] - 3s 53us/step - loss: 0.0418 - acc: 0.9874 - val_loss: 0.0290 - val_acc: 0.9910
Epoch 7/12
60000/60000 [==============================] - 3s 54us/step - loss: 0.0378 - acc: 0.9882 - val_loss: 0.0344 - val_acc

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.030294291781658468
Test accuracy: 0.99
